In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from manada import substructure
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.ImSim.image_model import ImageModel
from lenstronomy.SimulationAPI.data_api import DataAPI
from lenstronomy.Workflow.fitting_sequence import FittingSequence
from lenstronomy.LightModel.Profiles.shapelets import ShapeletSet
from lenstronomy.Data.psf import PSF
from lenstronomy.SimulationAPI.observation_api import SingleBand
from lenstronomy.Plots.model_plot import ModelPlot
import lenstronomy.Util.util as util
import lenstronomy.Util.image_util as image_util
import matplotlib.pyplot as plt
import os, corner, imageio, scipy
from ovejero import bnn_inference, model_trainer

# Modifies the paths in the config to agree with the paths being used on the current computer.
def recursive_str_checker(cfg_dict):
    for key in cfg_dict:
        if isinstance(cfg_dict[key],str):
            cfg_dict[key] = cfg_dict[key].replace('/home/swagnercarena/ovejero/',root_path)
        if isinstance(cfg_dict[key],dict):
            recursive_str_checker(cfg_dict[key])

# Understanding Main Deflector Modeling Residuals

__Author:__ Sebastian Wagner-Carena

__Goals:__ Understand what the residuals look like after attempting to fit a smooth lens model to a deflector that include substructure contributions. 

### Table of Contents

1. [Generating a Lens Image With Substructure](#lens_sub_basic) 
2. [The Danger of Simplistic Residual Estimates](#bias)
    1. [Simple Residual Estimate](#lens_sub_basi_resid)
    2. [Remodeling the Main Deflector](#fow_mod_remod)
    3. [Investigating Residuals With the Modeled Lens](#basic_to_fow_mod_comp)
3. [Generating Realistic Residuals](#realistic)
    1. [Removing Large Scale Deflection](#sharp_sub)
    2. [Replacing the Forward Modeling With a BNN](#BNN)
4. [Using a More Realistic Source](#real_source)
5. [Using a COSMOS Source](#cosmos)

One powerful data vector we have in the detection of substructure in strong lensing is the residual left after modeling away the smooth lens potential of the main deflector. In theory, the remaining signal can only be described by small scale, localized variations in the deflection field - a sign that there is substructure in our model. It is therefore tempting to use these residual maps as inputs to a machine learning based approach to substructure detection; in theory, this can even make your training agnostic to the exact lens model being subtracted. 

However, modeling and subtracting the main deflector can take dozens of CPU hours even with well optimized code. A tempting workaround is to produce two images: one that includes only the main deflector and one that includes the main deflector with its substrcuture. One can then subtract the main deflector only image from the substructure image and be left with what appears to be a reasonable residual. However, as we will see in these examples, this type of residual can produce a substantial training bias. With subtructure, __the true input lens is not the lens that would be returned by forward modeling__. The presence of substructure creates an overall increase in the total mass of the main deflector which is __degenerate with the einstein radius__ of the smooth lens model. 

In this notebook we will explore that degeneracy and show how one can create more realistic residual maps for the purposes of training.

## Generating a Lens Image With Substructure <a class="anchor" id="lens_sub_basic"></a>

The first step is to generate a lensing image with substructure. In order to best illustrate the dangers of the erronous approach to residual maps outlined above, we will use a fairly flexible lens model - the __power law elliptical mass distribution (PEMD)__. We will also include external shear.

For the substructure, we will (for now) use a very simple model. The substructure will all be __truncated NFW__ profiles with the parameters drawn either normally (in the case of mass and concentration parameters) or uniformly (in the case of parameters relating to the substructure's position in the halo). The number of substructure objects is fixed. 

In a similar vein, for the source we will use a simple __Sersic__ light profile. 

In [ ]:
# Set up each of our three models

# Setup the random seed so that the notebook always returns the same outputs. Also set up the numerics kwargs
seed=864
np.random.seed(seed)
kwargs_numerics = {'supersampling_factor':1}

# Parameters for our PEMD lens model
main_lens_model_list = ['PEMD','SHEAR']
kwargs_spemd = {'gamma': 1.96,'theta_E': 1.08, 'e1': -0.34, 'e2': 0.02, 'center_x': -0.05, 'center_y': 0.12}
kwargs_shear = {'gamma1': 0.05, 'gamma2': 0.02}
main_lens_kwargs_list = [kwargs_spemd,kwargs_shear]

# Parameters for our substructure drawn from the manada package
n_subs = 20
r_min, r_max = -3,2
alpha_rs_mean, alpha_rs_std = 0.05,0.05
rs_mean, rs_std = 0.05, 0.05
sub_model_list, sub_kwargs_list = substructure.substructure_realization(n_subs,r_max,r_min,rs_mean,rs_std,
                                                                        alpha_rs_mean,alpha_rs_std,
                                                                        kwargs_spemd['center_x'],
                                                                        kwargs_spemd['center_y'])

# Combine both into our current lens model
complete_lens_model_list = main_lens_model_list + sub_model_list
complete_lens_model_kwargs = main_lens_kwargs_list + sub_kwargs_list
complete_lens_model = LensModel(complete_lens_model_list)

# We're also going to generate a lens model without the substructure
simple_lens_model = LensModel(main_lens_model_list)

# Now build our source model
source_model_list = ['SERSIC_ELLIPSE']
source_kwargs_list = [{'R_sersic': 1.13, 'n_sersic': 1.42,'e1': 0.09, 'e2': -0.14, 'center_x': -0.02, 'center_y': -0.12}]
source_light_model = LightModel(source_model_list)

# Finally build our detector model (we have to add some realistic noise for our forward modeling later to
# work).
kwargs_psf = {'psf_type': 'GAUSSIAN', 'fwhm': 0.1}
psf_model = PSF(**kwargs_psf)
kwargs_detector = {'pixel_scale':0.08, 'ccd_gain':2.5, 'read_noise':4.0, 'magnitude_zero_point':25.9463, 
                   'exposure_time':5400.0, 'sky_brightness':22, 'num_exposures':1, 'background_noise':None}
numpix = 64  
data_api = DataAPI(numpix=numpix,**kwargs_detector)

# Turn our magnitude choice into an amplitude
mag = 22.1
# Computes the total surface brightness with amp = 1
cps_norm = source_light_model.total_flux(source_kwargs_list, norm=True, k=0)[0]
cps = data_api.magnitude2cps(mag)
amp = cps/ cps_norm
source_kwargs_list[0]['amp'] = amp


# Now we can make the image models for our two lensing systems (with and without substructure)
complete_image_model = ImageModel(data_api.data_class, psf_model, complete_lens_model, source_light_model, None, 
                                  None, kwargs_numerics=kwargs_numerics)
simple_image_model = ImageModel(data_api.data_class, psf_model, simple_lens_model, source_light_model, None, 
           None, kwargs_numerics=kwargs_numerics)
single_band = SingleBand(**kwargs_detector)

In [ ]:
complete_lens_model.lens_model_list

In [ ]:
# Now we can lens our light

# We first evaluate the source light at the beta coordinate and plot them at the alpha coordinates.
source_lensed_sub = complete_image_model.image(complete_lens_model_kwargs, source_kwargs_list, None, None)
source_lensed_sub_noise = source_lensed_sub+single_band.noise_for_model(source_lensed_sub)

# Now the model without substructure. Don't add noise since we want to understand the difference from a perfect
# signal 
source_lensed_simple = simple_image_model.image(main_lens_kwargs_list, source_kwargs_list, None, None)
source_lensed_simple_noise = source_lensed_simple+single_band.noise_for_model(source_lensed_simple)

## The Danger of Simplistic Residual Estimates <a class="anchor" id="bias"></a>

### Simple Residual Estimate <a class="anchor" id="lens_sub_basi_resid"></a>

Now we have one model with substructure and one without. A simple approximation would be to compare the lensed light from both images and call that our residual. Let's see what that looks like.

In [ ]:
# We can visiualize our lensed source in both cases
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)

# Plot the image with the substructure
im = ax[0].matshow(source_lensed_sub_noise, origin='lower')
ax[0].set_title("Lensed Source With Substructure")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the image without substructure
im = ax[1].matshow(source_lensed_simple, origin='lower')
ax[1].set_title("Lensed Source Without Substructure (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_sub_noise-source_lensed_simple, origin='lower')
f.colorbar(im)
ax[0].set_title("Simplistic Residual Estimate (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_sub-source_lensed_simple, origin='lower')
ax[1].set_title("Simplistic Residual Estimate (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

That looks like a big difference! There are some spatially concentrated residuals that clearly correspond to specific substructure, but it seems that on the larger scales our ring has also changed substantially between the two images. What's driving this change? 

Adding our 20 subhalos has also added mass to our lens, which means we will see changes in the __lensing at large and small scales__. In fact, most of our subhalos appear to only be impacting the lensing image through the large scale change, and only a few are leaving a visible small scall impact. The issue is that __large scale changes to our lensing potential from substructure are degenerate with the parameters of our main deflector__.

To illustrate this, let's go ahead and fit a lens model to the lensing image with substructure.

### Remodeling the Main Deflector  <a class="anchor" id="fow_mod_remod"></a>

Here we use the lenstronomy fitting sequence functions to forward model the PEMD parameters for the image of our lens with substructure.

In [ ]:
# The lists of all the model types and dicts of parameters
ls_lens_model_list = []
fixed_lens = []
kwargs_lens_init = []
kwargs_lens_sigma = []
kwargs_lower_lens = []
kwargs_upper_lens = []
ls_source_model_list = []
fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []

# Add initial values, boundaries, and spread for the PEMD parameters. No parameter needs to be kept fixed
ls_lens_model_list.append('PEMD')
fixed_lens.append({})
kwargs_lens_init.append({'theta_E': 1.08, 'e1': -0.34, 'e2': 0.02,'center_x': -0.05, 'center_y': 0.12, 'gamma': 1.96})
kwargs_lens_sigma.append({'theta_E': .2, 'e1': 0.05, 'e2': 0.05,'center_x': 0.05, 'center_y': 0.05, 'gamma': 0.2})
kwargs_lower_lens.append({'theta_E': 0.01, 'e1': -0.5, 'e2': -0.5,'center_x': -10, 'center_y': -10, 'gamma': 0.01})
kwargs_upper_lens.append({'theta_E': 10., 'e1': 0.5, 'e2': 0.5,'center_x': 10, 'center_y': 10, 'gamma': 10})

# Do the same for the shear. Fixing ra_0 and dec_0 for simplicity (i.e. not allowing the coordinate system to move).
ls_lens_model_list.append('SHEAR')
fixed_lens.append({'ra_0': 0, 'dec_0': 0})
kwargs_lens_init.append({'gamma1': 0.05, 'gamma2': 0.02})
kwargs_lens_sigma.append({'gamma1': 0.05, 'gamma2': 0.05})
kwargs_lower_lens.append({'gamma1': -10, 'gamma2': -10})
kwargs_upper_lens.append({'gamma1': 10, 'gamma2': 10})

# An finally for our source parameters. 
ls_source_model_list.append('SERSIC_ELLIPSE')
fixed_source.append({})
kwargs_source_init.append({'R_sersic': 1.13, 'n_sersic': 1.42,'e1': 0.09, 'e2': -0.14, 'center_x': -0.02, 
                               'center_y': -0.12})
kwargs_source_sigma.append({'n_sersic': 0.5, 'R_sersic': 0.1,'e1': 0.05, 'e2': 0.05, 'center_x': 0.2, 
                            'center_y': 0.2})
kwargs_lower_source.append({'e1': -0.5, 'e2': -0.5,'R_sersic': 0.001, 'n_sersic': .5, 'center_x': -10,
                            'center_y': -10})
kwargs_upper_source.append({'e1': 0.5, 'e2': 0.5, 'R_sersic': 10,'n_sersic': 5., 'center_x': 10, 'center_y': 10})

# Turn these into the list objects desired by lenstornomy
ls_lens_params = [kwargs_lens_init, kwargs_lens_sigma,fixed_lens, kwargs_lower_lens, kwargs_upper_lens]
ls_source_params = [kwargs_source_init, kwargs_source_sigma,fixed_source, kwargs_lower_source, kwargs_upper_source]
ls_kwargs_params = {'lens_model': ls_lens_params,'source_model': ls_source_params}
ls_kwargs_model = {'lens_model_list':ls_lens_model_list,'source_light_model_list':ls_source_model_list}

# Populate some of the observational and numerics kwargs we will need
ls_kwargs_likelihood = {'source_marg': False}
ls_kwargs_model = {'lens_model_list': ls_lens_model_list,'source_light_model_list': ls_source_model_list}
kwargs_numerics['supersampling_convolution']=False

# Now we can write in the image (with a few options specified for lenstronomy)
_, _, ra_0, dec_0, _, _, Mpix2coord, _ = util.make_grid_with_coordtransform(numPix=numpix,
                                                                            deltapix=kwargs_detector['pixel_scale'], 
                                                                            center_ra=0,center_dec=0,subgrid_res=1,
                                                                            inverse=False)

# This is where we specify that the image we want to conduct inference on is the lens WITH substructure
ls_kwargs_data = {'background_rms': single_band.background_noise,'exposure_time': single_band.exposure_time,
                  'ra_at_xy_0': ra_0,'dec_at_xy_0': dec_0,'transform_pix2angle': Mpix2coord,
                  'image_data': source_lensed_sub_noise}

# A few more lenstronomy options
ls_multi_band_list = [[ls_kwargs_data, kwargs_psf, kwargs_numerics]]
ls_kwargs_data_joint = {'multi_band_list': ls_multi_band_list,'multi_band_type': 'multi-linear'}

In [ ]:
# Now we initialize the fitting sequence (behind the scenes this uses emcee with parallelization)
ls_kwargs_constraints = {}
fitting_seq = FittingSequence(ls_kwargs_data_joint,ls_kwargs_model,ls_kwargs_constraints,
                              ls_kwargs_likelihood, ls_kwargs_params)

# And finally we can run our fitting sequence
walker_ratio = 10
# Set this to 5000+ if running the chains for the first time.
n_samps = 1
chains_save_path = 'MDMR_chains.hd5'
if os.path.isfile(chains_save_path):
    print('Using chains found at %s'%(chains_save_path))
    start_from_backup = True
else:
    print('No chains found at %s'%(chains_save_path))
    start_from_backup = False
fitting_kwargs_list = [['MCMC',{'n_burn': 0,'n_run': n_samps, 'walkerRatio': walker_ratio,'sigma_scale': 0.1, 
                                'backup_filename': chains_save_path,'start_from_backup': start_from_backup}]]
chain_list = fitting_seq.fit_sequence(fitting_kwargs_list)

In [ ]:
# Now let's take a look at the chains, and the fit
chain_params = chain_list[0][2]
# Get a mapping to original values of parameters
true_values = []
for param in chain_params:
    if 'lens0' in param:
        true_values.append(kwargs_spemd[param[:-6]])
    if 'lens1' in param:
        true_values.append(kwargs_shear[param[:-6]])
    if 'source_light0' in param:
        true_values.append(source_kwargs_list[0][param[:-14]])
chains = chain_list[0][1].reshape((-1,len(chain_params)*walker_ratio,len(chain_params)))

# Don't forget to include some burnin!
burnin = 4000
chains = chains[burnin:].reshape((-1,chains.shape[-1]))
plot_limits = None
color_contour = '#1b9e77'
truth_color = '#d95f02'
hist_kwargs = {'density':True,'color':color_contour}
fontsize = 13
dpi = 200
fig = corner.corner(chains,bins=20,labels=chain_params,show_titles=False,plot_datapoints=False,
                    label_kwargs=dict(fontsize=fontsize),truths=true_values,levels=[0.68,0.95],dpi=dpi, 
                    color=color_contour,fill_contours=True,range=plot_limits,truth_color=truth_color,
                    hist_kwargs=hist_kwargs)

It's clear that our emcee posterior has not converged to the correct values, but we expect this: the substructure is degenerate with parameters of our main deflector model. For example, the einstein radius is not estimated to be 1.01 rather than 1, corresponding nicely to the residual effect we plotted earlier. To further cemment this we can do the same comparison as before, but now between the lens light for the model our emcee converged to and the true model with substructure.

### Investigating Residuals With the Modeled Lens <a class="anchor" id="basic_to_fow_mod_comp"></a>

There are two questions we can ask ourselves about the residuals with the modeled lens.

1. Is there still a residual that is detectable above the noise?
2. Ignoring the noise, what does the residual actually look like?

In [ ]:
# We can pull all the kwargs from the mean of our chains after burnin.
kwargs_spemd_emcee = {}
kwargs_shear_emcee = {}
kwargs_source_emcee = {}
for pi, param in enumerate(chain_params):
    if 'lens0' in param:
        kwargs_spemd_emcee[param[:-6]] = np.mean(chains[:,pi])
    if 'lens1' in param:
        kwargs_shear_emcee[param[:-6]] = np.mean(chains[:,pi])
    if 'source_light0'  in param:
        kwargs_source_emcee[param[:-14]] = np.mean(chains[:,pi])

main_lens_kwargs_list_emcee = [kwargs_spemd_emcee,kwargs_shear_emcee]
main_lens_kwargs_source_emcee = [kwargs_source_emcee]

# Quickly scan through the amplitude to find the amplitude best fit
amps_fit = np.linspace(1,60,2000)
min_dif,amp_min = np.inf,0 
for amp in amps_fit:
    kwargs_source_emcee['amp'] = amp
    source_lensed_simple_emcee = simple_image_model.image(main_lens_kwargs_list_emcee, main_lens_kwargs_source_emcee, 
                                                None, None)
    square_dif = np.sum(np.square(source_lensed_simple_emcee-source_lensed_sub_noise))
    if square_dif < min_dif:
        min_dif = square_dif
        amp_min = amp
# Take the minimum value
kwargs_source_emcee['amp'] = amp_min 


# We can use the same model without substructure but just pass in the new parameters
source_lensed_simple_emcee = simple_image_model.image(main_lens_kwargs_list_emcee, main_lens_kwargs_source_emcee, 
                                                None, None)

In [ ]:
# We can visiualize our lensed source in both cases
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)

# Plot the image with the substructure
im = ax[0].matshow(source_lensed_sub_noise, origin='lower')
ax[0].set_title("Lensed Source With Substructure")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the image without substructure
im = ax[1].matshow(source_lensed_simple_emcee, origin='lower')
ax[1].set_title("Lensed Source Without Substructure (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_sub_noise-source_lensed_simple_emcee, origin='lower')
f.colorbar(im)
ax[0].set_title("True Residual Estimate (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_sub-source_lensed_simple_emcee, origin='lower')
ax[1].set_title("True Residual Estimate (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

That looks very different from the residual we calculated above! There's still signal there, but it's 3-4x weaker and therefore much harder to distinguish from the noise. But this is exactly the type of residual you would get if you conducted a forward model on the lens. If we train a network using the first set of residuals we showed, we would falsely belive that our network could easily detect the presence of substructure, when in reality it was simply detecting a change in the einstein ring caused by the inclusion of more mass.

Similar issues can arise even when not training a model on the residual; in generating a training set we have to be careful that the addition of the substructure __does not produce signal simply because our assumptions about the lens profile were too simplistic__!

## Generating Realistic Residuals <a class="anchor" id="realistic"></a>

### Removing Large Scale Deflection <a class="anchor" id="sharp_sub"></a>

One potentially cheap and rapid way to create residuals is to remove the large scale signal in the deflection angle. In theory this is equivalent to fitting the large scale defelction angle contribution with the smoothed mass model. Of course there are some potential issues here.

1. The smooth models we use (PEMD) have power at all scales so it's hard to determine an exact smoothing scale where the contribution from the large scale potential should no longer be dominant.
2. The large scale contribution from the substructure changes the Einstein radius and therefore changes the location of the lensing signal. This means that the location of the small scale contribution of the substructure also changes. This may not be an issue if all we want to do is substructure inference, but it's important to look out for.

Regardless, let's give this approach a try here and see how it compares to the residuals we got from directly forward modeling.

In [ ]:
# First we need to create a lens model that has only the substructure. From there we can get the substructure 
# deflection and remove the large scale contribution.
sub_only_lens_model = LensModel(sub_model_list)

# Get the grid we're going to evaluate our deflection angle and Hessian on and conduct the smoothing
x_grid, y_grid = util.make_grid(numPix=numpix, deltapix=kwargs_detector['pixel_scale'])
f = sub_only_lens_model.potential(x_grid, y_grid,sub_kwargs_list)
a_x, a_y = sub_only_lens_model.alpha(x_grid, y_grid, sub_kwargs_list)
a_x, a_y = sub_only_lens_model.alpha(x_grid, y_grid, sub_kwargs_list)
f_xx, f_xy, f_yx, f_yy = sub_only_lens_model.hessian(x_grid, y_grid, sub_kwargs_list)
sigma = 7
f_s = util.array2image(f)- scipy.ndimage.gaussian_filter(util.array2image(f),sigma=sigma)
a_x_s = util.array2image(a_x)-scipy.ndimage.gaussian_filter(util.array2image(a_x),sigma=sigma)
a_y_s = util.array2image(a_y)-scipy.ndimage.gaussian_filter(util.array2image(a_y),sigma=sigma)
f_xx_s = util.array2image(f_xx)-scipy.ndimage.gaussian_filter(util.array2image(f_xx),sigma=sigma)
f_yy_s = util.array2image(f_yy)-scipy.ndimage.gaussian_filter(util.array2image(f_yy),sigma=sigma)
f_xy_s = util.array2image(f_xy)-scipy.ndimage.gaussian_filter(util.array2image(f_xy),sigma=sigma)

# Now we can create a lens model that only has this smale scale component of the deflection angle
no_smooth_lens_model_list = main_lens_model_list + ['INTERPOL']
no_smooth_lens_model = LensModel(no_smooth_lens_model_list)
x_axes, y_axes = util.get_axes(x_grid, y_grid)
no_smooth_lens_model_kwargs = [{'grid_interp_x':x_axes, 'grid_interp_y':y_axes, 'f_':f_s, 
                                'f_x':a_x_s, 'f_y':a_y_s, 'f_xx':f_xx_s, 'f_yy':f_yy_s, 'f_xy':f_xy_s}]
no_smooth_lens_model_kwargs = main_lens_kwargs_list + no_smooth_lens_model_kwargs


# Finaly we can create the corresponding image model and image,
no_smooth_lens_model = ImageModel(data_api.data_class, psf_model, no_smooth_lens_model, source_light_model, 
                                  None, None, kwargs_numerics=kwargs_numerics)
# Now we can lens our light
no_smooth_lensed = no_smooth_lens_model.image(no_smooth_lens_model_kwargs, source_kwargs_list, None, None)
no_smooth_lensed_noise = no_smooth_lensed+single_band.noise_for_model(no_smooth_lensed)

In [ ]:
# We can visiualize our lensed source in both cases
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)

# Plot the image with the substructure
im = ax[0].matshow(no_smooth_lensed_noise, origin='lower')
ax[0].set_title("No Large Scale Substructure Deflection")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the image without substructure
im = ax[1].matshow(source_lensed_simple, origin='lower')
ax[1].set_title("Lensed Source Without Substructure (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(no_smooth_lensed_noise-source_lensed_simple, origin='lower')
f.colorbar(im)
ax[0].set_title("Residual (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(no_smooth_lensed-source_lensed_simple, origin='lower')
ax[1].set_title("Residual (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_sub_noise-source_lensed_simple_emcee, origin='lower')
f.colorbar(im)
ax[0].set_title("True Residual Estimate (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_sub-source_lensed_simple_emcee, origin='lower')
ax[1].set_title("True Residual Estimate (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

Not a perfect match, but clearly this gets us much closer to the true residual signal we would get from properly forward modeling. The question then is, is this close enough? We could do a lot of analysis of the two residuals to try to answer that question, but the simplest way is to train a model using the estimated residuals and see how it performs when we expose it to the true residuals.

### Replacing the Forward Modeling With a BNN <a class="anchor" id="BNN"></a>

One way we could imagine creating realistic residuals quickly is by using a BNN model to predict the parameters of our PEMD (assuming no contribution from the substructure). There are a few reasons we may want to be cautios with an approach like this however:

1. The BNN has been trained assuming no substructure and now being asked to do inference on images with substructure. In theory that should return something close to the PEMD output we get from forward modeling, but as we've shown in our previous work with ovejero it can be dangerous to push a BNN outside the boundaires of its training set.

2. One of the big takeaways from BNN work on strong lensing is that it's important to be able to do hierarchical inference in order to correct for the training set bias. This becomes much dicier when the BNN is only the first step in your inference.
    
3. It becomes nearly impossible to check the calibration of the BNN. We know it should be predicting parameter values that aren't the true values (as we demonstrated above the substructure should have a meaningful effect on the smooth distribution), but before running forward modeling we don't know what those shifted predictions should be. Without access to the ground truth of a test set, it becomes impossible to make a statement about statistical consistency of our posterior.

With those caveats clarified, we can see what kind of results we get.

In [ ]:
# We will use ovejero to run our network. on the image we've simulated.
# First we need to load the config.
root_path = os.getcwd()[:-9]
config_path = root_path + 'configs/ovejero/nn3_slr.json'
bnn_cfg = model_trainer.load_config(config_path)
recursive_str_checker(bnn_cfg)

# Now we use the config to intialize our BNN inference model.
bnn_infer = bnn_inference.InferenceClass(bnn_cfg,load_val_dataset=False)
bnn_infer_no_sub = bnn_inference.InferenceClass(bnn_cfg,load_val_dataset=False)

# And now we can run our BNN on our image
num_samples = 10000
sample_save_dir = 'bnn_samples_MDMR'
bnn_infer.gen_samples(num_samples,sample_save_dir=sample_save_dir,
                      single_image=source_lensed_sub_noise/np.std(source_lensed_sub_noise))

# We can also run our BNN on the image without substructure
sample_save_dir = 'bnn_samples_MDMR_no_sub'
bnn_infer_no_sub.gen_samples(num_samples,sample_save_dir=sample_save_dir,
                      single_image=source_lensed_simple_noise/np.std(source_lensed_simple_noise))

# Let's take a look at the BNN samples
corner_bnn_samples = bnn_infer.predict_samps.reshape(-1,bnn_infer.predict_samps.shape[-1])
corner_bnn_samples_no_sub = bnn_infer_no_sub.predict_samps.reshape(-1,bnn_infer_no_sub.predict_samps.shape[-1])
contour_color = 'k'
hist_kwargs = {'density':True,'color':contour_color}
fig = corner.corner(corner_bnn_samples,bins=20,labels=bnn_infer.final_params_print_names,show_titles=False,
                    plot_datapoints=False,label_kwargs=dict(fontsize=fontsize),levels=[0.68,0.95],
                    color=contour_color,fill_contours=True,hist_kwargs=hist_kwargs)

In [ ]:
# We can pull all the kwargs from the mean of our BNN. Since the BNN doesn't predict the sersic parameters, 
# we'll have to fit those
kwargs_spemd_bnn = {}
kwargs_shear_bnn = {}
for pi, param in enumerate(bnn_infer.final_params):
    if 'lens_mass' in param:
        kwargs_spemd_bnn[param[10:]] = np.median(corner_bnn_samples[:,pi])
    if 'external_shear' in param:
        kwargs_shear_bnn['gamma'+param[-1:]] = np.median(corner_bnn_samples[:,pi])

kwargs_spemd_bnn['theta_E'] = np.exp(kwargs_spemd_bnn.pop('theta_E_log'))
main_lens_kwargs_list_bnn = [kwargs_spemd_bnn,kwargs_shear_bnn]

# Repeat the same for the model without substructure.
kwargs_spemd_bnn_no_sub = {}
kwargs_shear_bnn_no_sub = {}
for pi, param in enumerate(bnn_infer_no_sub.final_params):
    if 'lens_mass' in param:
        kwargs_spemd_bnn_no_sub[param[10:]] = np.median(corner_bnn_samples_no_sub[:,pi])
    if 'external_shear' in param:
        kwargs_shear_bnn_no_sub['gamma'+param[-1:]] = np.median(corner_bnn_samples_no_sub[:,pi])

kwargs_spemd_bnn_no_sub['theta_E'] = np.exp(kwargs_spemd_bnn_no_sub.pop('theta_E_log'))
main_lens_kwargs_list_bnn = [kwargs_spemd_bnn,kwargs_shear_bnn]
main_lens_kwargs_list_bnn_no_sub = [kwargs_spemd_bnn_no_sub,kwargs_shear_bnn_no_sub]

In [ ]:
# The lists of all the model types and dicts of parameters
bnn_ls_lens_model_list = []
bnn_fixed_lens = []
bnn_kwargs_lens_init = [{},{}]
bnn_kwargs_lens_sigma = [{},{}]
bnn_kwargs_lower_lens = [{},{}]
bnn_kwargs_upper_lens = [{},{}]
bnn_ls_source_model_list = []
bnn_fixed_source = []
bnn_kwargs_source_init = []
bnn_kwargs_source_sigma = []
bnn_kwargs_lower_source = []
bnn_kwargs_upper_source = []

# Fix all the lens values
bnn_ls_lens_model_list.append('PEMD')
bnn_fixed_lens.append(kwargs_spemd_bnn)

# Do the same for the shear. Fixing ra_0 and dec_0 for simplicity (i.e. not allowing the coordinate system to move).
bnn_ls_lens_model_list.append('SHEAR')
# Join the two dicts
bnn_fixed_lens.append({**{'ra_0': 0, 'dec_0': 0},**kwargs_shear_bnn})

# An finally for our source parameters. 
bnn_ls_source_model_list.append('SERSIC_ELLIPSE')
bnn_fixed_source.append({})
bnn_kwargs_source_init.append({'R_sersic': 1.13, 'n_sersic': 1.42,'e1': 0.09, 'e2': -0.14, 'center_x': -0.02, 
                               'center_y': -0.12})
bnn_kwargs_source_sigma.append({'n_sersic': 0.5, 'R_sersic': 0.1,'e1': 0.05, 'e2': 0.05, 'center_x': 0.2, 
                            'center_y': 0.2})
bnn_kwargs_lower_source.append({'e1': -0.5, 'e2': -0.5,'R_sersic': 0.001, 'n_sersic': .5, 'center_x': -10,
                            'center_y': -10})
bnn_kwargs_upper_source.append({'e1': 0.5, 'e2': 0.5, 'R_sersic': 10,'n_sersic': 5., 'center_x': 10, 'center_y': 10})

# Turn these into the list objects desired by lenstornomy
bnn_ls_lens_params = [bnn_kwargs_lens_init, bnn_kwargs_lens_sigma, bnn_fixed_lens, bnn_kwargs_lower_lens, 
                      bnn_kwargs_upper_lens]
bnn_ls_source_params = [bnn_kwargs_source_init, bnn_kwargs_source_sigma, bnn_fixed_source, bnn_kwargs_lower_source, 
                        bnn_kwargs_upper_source]
bnn_ls_kwargs_params = {'lens_model': bnn_ls_lens_params,'source_model': bnn_ls_source_params}
bnn_ls_kwargs_model = {'lens_model_list':bnn_ls_lens_model_list,'source_light_model_list':bnn_ls_source_model_list}

# Populate some of the observational and numerics kwargs we will need
bnn_ls_kwargs_model = {'lens_model_list': bnn_ls_lens_model_list,'source_light_model_list': bnn_ls_source_model_list}

In [ ]:
# Now we initialize the fitting sequence (behind the scenes this uses emcee with parallelization)
bnn_fitting_seq = FittingSequence(ls_kwargs_data_joint,bnn_ls_kwargs_model,ls_kwargs_constraints,
                              ls_kwargs_likelihood, bnn_ls_kwargs_params)

# And finally we can run our fitting sequence
walker_ratio = 10
# Set this to 3000+ if running chains for the first time.
n_samps = 1
bnn_chains_save_path = 'MDMR_bnn_source_chains.hd5'
if os.path.isfile(bnn_chains_save_path):
    print('Using chains found at %s'%(bnn_chains_save_path))
    start_from_backup = True
else:
    print('No chains found at %s'%(bnn_chains_save_path))
    start_from_backup = False
fitting_kwargs_list = [['MCMC',{'n_burn': 0,'n_run': n_samps, 'walkerRatio': walker_ratio,'sigma_scale': 0.1, 
                                'backup_filename': bnn_chains_save_path,'start_from_backup': start_from_backup}]]
bnn_chain_list = bnn_fitting_seq.fit_sequence(fitting_kwargs_list)

# Now let's take a look at the chains, and the fit
bnn_chain_params = bnn_chain_list[0][2]
bnn_chains = bnn_chain_list[0][1].reshape((-1,len(bnn_chain_params)*walker_ratio,len(bnn_chain_params)))

# Don't forget to include some burnin!
burnin = 2500
bnn_chains = bnn_chains[burnin:].reshape((-1,bnn_chains.shape[-1]))

kwargs_source_bnn = {}
for pi, param in enumerate(bnn_chain_params):
    if 'source_light0'  in param:
        kwargs_source_bnn[param[:-14]] = np.mean(bnn_chains[:,pi])
        
# Quickly scan through the amplitude to find the amplitude best fit
amps_fit = np.linspace(1,10,2000)
min_dif,amp_min = np.inf,0 
for amp in amps_fit:
    kwargs_source_bnn['amp'] = amp
    source_lensed_bnn = simple_image_model.image(main_lens_kwargs_list_bnn, [kwargs_source_bnn], 
                                                None, None)
    square_dif = np.sum(np.square(source_lensed_bnn-source_lensed_sub_noise))
    if square_dif < min_dif:
        min_dif = square_dif
        amp_min = amp
# Take the minimum value
kwargs_source_bnn['amp'] = amp_min
        
# We can use the same model without substructure but just pass in the new parameters
source_lensed_bnn = simple_image_model.image(main_lens_kwargs_list_bnn, [kwargs_source_bnn], 
                                                None, None)

In [ ]:
# Repeat the same for the model without substructure 
bnn_fixed_lens_no_sub = []
bnn_fixed_lens_no_sub.append(kwargs_spemd_bnn_no_sub)
bnn_fixed_lens_no_sub.append({**{'ra_0': 0, 'dec_0': 0},**kwargs_shear_bnn_no_sub})

# Turn these into the list objects desired by lenstornomy
bnn_ls_lens_params_no_sub = [bnn_kwargs_lens_init, bnn_kwargs_lens_sigma, bnn_fixed_lens_no_sub, 
                             bnn_kwargs_lower_lens, bnn_kwargs_upper_lens]
bnn_ls_kwargs_params_no_sub = {'lens_model': bnn_ls_lens_params_no_sub,'source_model': bnn_ls_source_params}

# Now we initialize the fitting sequence (behind the scenes this uses emcee with parallelization)
bnn_fitting_seq_no_sub = FittingSequence(ls_kwargs_data_joint,bnn_ls_kwargs_model,ls_kwargs_constraints,
                              ls_kwargs_likelihood, bnn_ls_kwargs_params_no_sub)

# And finally we can run our fitting sequence
walker_ratio = 10
# Set this to 3000+ if running chains for the first time.
n_samps = 1
bnn_chains_save_path = 'MDMR_bnn_source_chains_no_sub.hd5'
if os.path.isfile(bnn_chains_save_path):
    print('Using chains found at %s'%(bnn_chains_save_path))
    start_from_backup = True
else:
    print('No chains found at %s'%(bnn_chains_save_path))
    start_from_backup = False
fitting_kwargs_list_no_sub = [['MCMC',{'n_burn': 0,'n_run': n_samps, 'walkerRatio': walker_ratio,'sigma_scale': 0.1, 
                                'backup_filename': bnn_chains_save_path,'start_from_backup': start_from_backup}]]
bnn_chain_list_no_sub = bnn_fitting_seq_no_sub.fit_sequence(fitting_kwargs_list_no_sub)

# Now let's take a look at the chains, and the fit
bnn_chain_params_no_sub = bnn_chain_list_no_sub[0][2]
bnn_chains_no_sub = bnn_chain_list_no_sub[0][1].reshape((-1,len(bnn_chain_params_no_sub)*walker_ratio,
                                                         len(bnn_chain_params_no_sub)))

# Don't forget to include some burnin!
burnin = 2500
bnn_chains_no_sub = bnn_chains_no_sub[burnin:].reshape((-1,bnn_chains_no_sub.shape[-1]))

kwargs_source_bnn_no_sub = {}
for pi, param in enumerate(bnn_chain_params_no_sub):
    if 'source_light0'  in param:
        kwargs_source_bnn_no_sub[param[:-14]] = np.mean(bnn_chains_no_sub[:,pi])
        
# Quickly scan through the amplitude to find the amplitude best fit
amps_fit = np.linspace(1,10,2000)
min_dif,amp_min = np.inf,0 
for amp in amps_fit:
    kwargs_source_bnn_no_sub['amp'] = amp
    source_lensed_bnn_no_sub = simple_image_model.image(main_lens_kwargs_list_bnn_no_sub, [kwargs_source_bnn_no_sub], 
                                                        None, None)
    square_dif = np.sum(np.square(source_lensed_bnn_no_sub-source_lensed_sub_noise))
    if square_dif < min_dif:
        min_dif = square_dif
        amp_min = amp
# Take the minimum value
kwargs_source_bnn_no_sub['amp'] = amp_min
        
# We can use the same model without substructure but just pass in the new parameters
source_lensed_bnn_no_sub = simple_image_model.image(main_lens_kwargs_list_bnn_no_sub, [kwargs_source_bnn_no_sub], 
                                                    None, None)

We want to compare the residuals we get from the BNN to the residuals without substructure (to understand if the residual signal just comes from the BNN not being as accurate a tool for fitting as the forward model) and to the forward modeling residuals themselves.

In [ ]:
# We can visiualize our lensed source in both cases
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)

# Plot the image with the substructure
im = ax[0].matshow(source_lensed_sub_noise, origin='lower')
f.colorbar(im)
ax[0].set_title("Lensed Source With Substructure")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the image without substructure
im = ax[1].matshow(source_lensed_bnn, origin='lower')
ax[1].set_title("Lensed Source Without Substructure (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_sub_noise-source_lensed_bnn, origin='lower')
f.colorbar(im)
ax[0].set_title("BNN Residual Estimate (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_sub-source_lensed_bnn, origin='lower')
ax[1].set_title("BNN Residual Estimate (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_sub_noise-source_lensed_simple_emcee, origin='lower')
f.colorbar(im)
ax[0].set_title("True Residual Estimate (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_sub-source_lensed_simple_emcee, origin='lower')
ax[1].set_title("True Residual Estimate (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual for modeling without substructure
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_simple_noise-source_lensed_bnn_no_sub, origin='lower')
f.colorbar(im)
ax[0].set_title("No Substructure Residual Estimate (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_simple-source_lensed_bnn_no_sub, origin='lower')
ax[1].set_title("No Substructure Residual Estimate (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

We see two things here. First, the residuals returned by the BNN are not far from those returned by the forward modeling. They contain essentially the same structure, and they are about as bright. Also, when we compare to the BNN run on a system without substructure (where the residuals could theoretically be zero), the BNN mean returns very small residuals compared to the substructure residuals. This is also important: there is a meaningful difference between the residuals from our BNNs slightly imprecise modeling and those returned by the presence of substructure. Again, as with the residuals without large scale deflection in the substructure, the best way to test if this is good enough is to train a model using these residuals and then see how it performs when exposed to lenses that have been properly forward modeled.

However, there is one interesting test we can perform. We can take multiple BNN posteriors samples and see how the different residuals compare. What we want to see is that the variance in the residual is subdominant to the signal.

In [ ]:
# Save our residuals
n_posterior_samples = 5
save_image = np.zeros((n_posterior_samples,)+source_lensed_sub_noise.shape)

for n_pos in range(n_posterior_samples):
    print('bnn sample %d'%(n_pos))
    # Redo the fitting
    bnn_samp = np.random.randint(corner_bnn_samples.shape[0])
    kwargs_spemd_bnn = {}
    kwargs_shear_bnn = {}
    for pi, param in enumerate(bnn_infer.final_params):
        if 'lens_mass' in param:
            kwargs_spemd_bnn[param[10:]] = corner_bnn_samples[bnn_samp,pi]
        if 'external_shear' in param:
            kwargs_shear_bnn['gamma'+param[-1:]] = corner_bnn_samples[bnn_samp,pi]

    kwargs_spemd_bnn['theta_E'] = np.exp(kwargs_spemd_bnn.pop('theta_E_log'))
    main_lens_kwargs_list_bnn = [kwargs_spemd_bnn,kwargs_shear_bnn]
    
    # Fix all the lens values
    bnn_fixed_lens = []
    bnn_fixed_lens.append(kwargs_spemd_bnn)

    # Do the same for the shear.
    # Join the two dicts
    bnn_fixed_lens.append({**{'ra_0': 0, 'dec_0': 0},**kwargs_shear_bnn})

    # Turn these into the list objects desired by lenstornomy
    bnn_ls_lens_params = [bnn_kwargs_lens_init, bnn_kwargs_lens_sigma, bnn_fixed_lens, bnn_kwargs_lower_lens, 
                          bnn_kwargs_upper_lens]
    bnn_ls_kwargs_params = {'lens_model': bnn_ls_lens_params,'source_model': bnn_ls_source_params}
    
    # Now we initialize the fitting sequence (behind the scenes this uses emcee with parallelization)
    bnn_fitting_seq = FittingSequence(ls_kwargs_data_joint,bnn_ls_kwargs_model,ls_kwargs_constraints,
                                  ls_kwargs_likelihood, bnn_ls_kwargs_params)
    n_particles = 100
    n_iterations = 100
    fitting_kwargs_list = [['PSO', {'sigma_scale': 1., 'n_particles': n_particles, 'n_iterations': n_iterations}]]
    bnn_chain_list = bnn_fitting_seq.fit_sequence(fitting_kwargs_list)
    kwargs_source_bnn = bnn_fitting_seq.best_fit()['kwargs_source']
    
    amps_fit = np.linspace(1,10,200)
    min_dif,amp_min = np.inf,0 
    for amp in amps_fit:
        kwargs_source_bnn[0]['amp'] = amp
        source_lensed_bnn = simple_image_model.image(main_lens_kwargs_list_bnn, kwargs_source_bnn, 
                                                    None, None)
        square_dif = np.sum(np.square(source_lensed_bnn-source_lensed_sub_noise))
        if square_dif < min_dif:
            min_dif = square_dif
            amp_min = amp
    # Take the minimum value
    kwargs_source_bnn[0]['amp'] = amp_min

    # We can use the same model without substructure but just pass in the new parameters
    source_lensed_bnn = simple_image_model.image(main_lens_kwargs_list_bnn, kwargs_source_bnn, 
                                                    None, None)
    save_image[n_pos]=source_lensed_bnn

In [ ]:
for n_pos in range(n_posterior_samples):
    # Plot the residual
    f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
    im = ax[0].matshow(source_lensed_sub_noise-save_image[n_pos], origin='lower',vmin=-0.03,vmax=0.03)
    f.colorbar(im)
    ax[0].set_title("Residual Estimate %d"%(n_pos+1))
    ax[0].get_xaxis().set_visible(False)
    ax[0].get_yaxis().set_visible(False)
    ax[0].autoscale(False)

    # Plot the residual no noise
    im = ax[1].matshow(source_lensed_sub-save_image[n_pos], origin='lower',vmin=-0.03,vmax=0.03)
    ax[1].set_title("Residual Estimate %d (No Noise)"%(n_pos+1))
    ax[1].get_xaxis().set_visible(False)
    ax[1].get_yaxis().set_visible(False)
    ax[1].autoscale(False)
    plt.show()

We can compare the variance in the residual to the strength of the residual signal itself.

In [ ]:
residuals = source_lensed_sub-save_image
plt.imshow(np.std(residuals,axis=0),origin='lower')
plt.colorbar()

## Using a More Realistic Source <a class="anchor" id="real_source"></a>

While the examples above show the pitfalls of not remodeling the main deflector, we are still being conservative in the choices being made. Primarily, we have fixed the source model to be a sersice function. A more flexible source model (which would be essential for modeling realsitic sources) could have further degeneracies with the residuals being measured. To illustrate this, we will redo the above analysis with a realistic source galaxy and shapelets for the source modeling.

In [ ]:
# Start by fitting a realistic source using shapelets.
ngc_filename = os.path.join('ngc1300.jpg')
# Read data, this only works if you execute this notebook within the environment of the github repo!
ngc_data = imageio.imread(ngc_filename, as_gray=True, pilmode=None)

# Subtract the median of an edge of the image
median = np.median(ngc_data[:200, :200])
ngc_data -= median

# Resize the image to square size (add zeros at the edges of the non-square bits of the image)
nx, ny = np.shape(ngc_data)
n_min = min(nx, ny)
n_max = max(nx, ny)
ngc_square = np.zeros((n_max, n_max))
x_start = int((n_max - nx)/2.)
y_start = int((n_max - ny)/2.)
ngc_square[x_start:x_start+nx, y_start:y_start+ny] = ngc_data

# Convolve the image with a Gaussian convolution kernel of a few pixels
sigma = 5
ngc_conv = scipy.ndimage.filters.gaussian_filter(ngc_square, sigma, mode='nearest', truncate=6)

# Degrade the data
factor = 25 
numPix_large = int(len(ngc_conv)/factor)
n_new = int((numPix_large-1)*factor)
ngc_cut = ngc_conv[0:n_new,0:n_new]
x, y = util.make_grid(numPix=numPix_large-1, deltapix=1)  
ngc_data_resized = image_util.re_size(ngc_cut, factor)

# Turn the image in a single 1d array for shapelets
image_1d = util.image2array(ngc_data_resized) 

# Specify our shapelet parameters
n_max = 100 
beta = 10

# Decompose image and return the shapelet coefficients
shapeletSet = ShapeletSet()
coeff_ngc = shapeletSet.decomposition(image_1d, x, y, n_max, beta, 1., center_x=0, center_y=0)
image_reconstructed = shapeletSet.function(x, y, coeff_ngc, n_max, beta, center_x=0, center_y=0)

# View our 2D Image
image_reconstructed_2d = util.array2image(image_reconstructed)
plt.imshow(image_reconstructed_2d,origin='lower')
plt.title('Shapelet Source')
plt.colorbar()
plt.show()

In [ ]:
# Now we can create our lensed image as before, but using this new source
# Now build our source model
shapelet_model_list = ['SHAPELETS']

# We'll chose a different size for the source and set the amplitude to roughly match our desired magnitude
source_beta = 0.05
# For now I'm just chosing this by hand so the brightness is similar
amp_coeff = coeff_ngc / 8
shapelet_kwargs_list = [{'n_max': n_max, 'beta': source_beta, 'center_x': 0.0, 'center_y': 0.0, 'amp':amp_coeff}]
shapelet_light_model = LightModel(shapelet_model_list)

# Now we can make the image model
shapelet_image_model = ImageModel(data_api.data_class, psf_model, complete_lens_model, shapelet_light_model, None, 
                                  None, kwargs_numerics=kwargs_numerics)
shapelet_image_model_no_sub = ImageModel(data_api.data_class, psf_model, simple_lens_model, shapelet_light_model, 
                                         None, None, kwargs_numerics=kwargs_numerics)

# And finally we can generate the image
source_lensed_shape = shapelet_image_model.image(complete_lens_model_kwargs, shapelet_kwargs_list, None, None)
source_lensed_shape_noise = source_lensed_shape+single_band.noise_for_model(source_lensed_shape)
source_lensed_shape_no_sub = shapelet_image_model_no_sub.image(main_lens_kwargs_list, shapelet_kwargs_list, 
                                                               None, None)

As before, we want to fit a model to this lens, but now with our shapelet source.

In [ ]:
# The lists of all the shapelet parameters
ls_shape_source_model_list = []
fixed_shape_source = []
kwargs_shape_source_init = []
kwargs_shape_source_sigma = []
kwargs_shape_lower_source = []
kwargs_shape_upper_source = []

# Add our shapelet parameters. 
ls_shape_source_model_list.append('SHAPELETS')
# This needs to be tuned to be reasonable to the precision required.
n_max_fit = 10
fixed_shape_source.append({'n_max': n_max_fit})
kwargs_shape_source_init.append({'beta': source_beta, 'center_x': 0.0, 'center_y': 0.0})
kwargs_shape_source_sigma.append({'beta': 0.01, 'center_x': 0.01, 'center_y': 0.01})
kwargs_shape_lower_source.append({'beta': 0.01, 'center_x': -0.1, 'center_y': -0.1})
kwargs_shape_upper_source.append({'beta': 0.1, 'center_x': 0.1, 'center_y': 0.1})

ls_shape_source_params = [kwargs_shape_source_init, kwargs_shape_source_sigma,fixed_shape_source, 
                          kwargs_shape_lower_source, kwargs_shape_upper_source]
ls_shape_kwargs_params = {'lens_model': ls_lens_params,'source_model': ls_shape_source_params}
ls_shape_kwargs_model = {'lens_model_list':ls_lens_model_list,'source_light_model_list':ls_shape_source_model_list}

# Need to redo this since there's now a new image
ls_shape_kwargs_data = {'background_rms': single_band.background_noise,'exposure_time': single_band.exposure_time,
                        'ra_at_xy_0': ra_0 ,'dec_at_xy_0': dec_0,'transform_pix2angle': Mpix2coord,
                        'image_data': source_lensed_shape_noise}

# A few more lenstronomy options
ls_shape_multi_band_list = [[ls_shape_kwargs_data, kwargs_psf, kwargs_numerics]]
ls_shape_kwargs_data_joint = {'multi_band_list': ls_shape_multi_band_list,'multi_band_type': 'multi-linear'}

fitting_seq_shape = FittingSequence(ls_shape_kwargs_data_joint,ls_shape_kwargs_model,ls_kwargs_constraints,
                                    ls_kwargs_likelihood, ls_shape_kwargs_params)

# And finally we can run our fitting sequence
n_particles = 200
n_iterations = 200
    
fitting_kwargs_list = [['PSO', {'sigma_scale': 1., 'n_particles': n_particles, 'n_iterations': n_iterations}]]
chain_list = fitting_seq_shape.fit_sequence(fitting_kwargs_list)
kwargs_result = fitting_seq_shape.best_fit()

# Now go back and plug in the lens parameter fit for n_max of 10 and run it on n_max of 30
n_max_fit = 25
fixed_shape_source[0]['n_max'] = n_max_fit

fixed_shape_lens = kwargs_result['kwargs_lens']
kwargs_shape_lens_init = [{},{}]
kwargs_shape_lens_sigma = [{},{}]
kwargs_shape_lower_lens = [{},{}]
kwargs_shape_upper_lens = [{},{}]

ls_shape_lens_params = [kwargs_shape_lens_init, kwargs_shape_lens_sigma, fixed_shape_lens, 
                        kwargs_shape_lower_lens, kwargs_shape_upper_lens]
ls_shape_kwargs_params = {'lens_model': ls_shape_lens_params,'source_model': ls_shape_source_params}
ls_shape_kwargs_model = {'lens_model_list':ls_lens_model_list,'source_light_model_list':ls_shape_source_model_list}

# A few more lenstronomy options
ls_shape_multi_band_list = [[ls_shape_kwargs_data, kwargs_psf, kwargs_numerics]]
ls_shape_kwargs_data_joint = {'multi_band_list': ls_shape_multi_band_list,'multi_band_type': 'multi-linear'}

fitting_seq_shape = FittingSequence(ls_shape_kwargs_data_joint,ls_shape_kwargs_model,ls_kwargs_constraints,
                                    ls_kwargs_likelihood, ls_shape_kwargs_params)

# And finally we can run our fitting sequence
n_particles = 30
n_iterations = 30
    
fitting_kwargs_list = [['PSO', {'sigma_scale': 1., 'n_particles': n_particles, 'n_iterations': n_iterations}]]
chain_list = fitting_seq_shape.fit_sequence(fitting_kwargs_list)

In [ ]:
new_kwargs_result = fitting_seq_shape.best_fit()
source_lensed_shape_recon = shapelet_image_model_no_sub.image(new_kwargs_result['kwargs_lens'], 
                                                              new_kwargs_result['kwargs_source'],None, None)
modelPlot = ModelPlot(ls_shape_multi_band_list, ls_shape_kwargs_model,new_kwargs_result)
    
f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

modelPlot.data_plot(ax=axes[0,0])
modelPlot.model_plot(ax=axes[0,1])
modelPlot.normalized_residual_plot(ax=axes[0,2], v_min=-6, v_max=6)
modelPlot.source_plot(ax=axes[1, 0], deltaPix_source=0.01, numPix=100)
modelPlot.convergence_plot(ax=axes[1, 1], v_max=1)
modelPlot.magnification_plot(ax=axes[1, 2])
f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0., hspace=0.05)
plt.show()

Compare that to the residuals from subtracting the ground truth without substructure from the noisy model with substructure and you can see that a lot of the signal has been washed away!

In [ ]:
# Plot the residual from the true model (i.e. without refitting)
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_shape_noise-source_lensed_shape_no_sub, origin='lower')
f.colorbar(im)
ax[0].set_title("Simpmlistic Residual (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_shape-source_lensed_shape_no_sub, origin='lower')
ax[1].set_title("Simplistic Residual (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual with refitting
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_shape_noise-source_lensed_shape_recon, origin='lower')
f.colorbar(im)
ax[0].set_title("Fit Residual (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_shape-source_lensed_shape_recon, origin='lower')
ax[1].set_title("Fit Residual (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

As before, we can make the comparison to the model with the large scale contribution to the deflection removed. We do not have a BNN that runs on images with more complex sources pre-trained.

In [ ]:
# We just have to create a model without 
no_smooth_lens_model = LensModel(no_smooth_lens_model_list)
no_smooth_lens_model_shape = ImageModel(data_api.data_class, psf_model, no_smooth_lens_model, shapelet_light_model, 
                                  None, None, kwargs_numerics=kwargs_numerics)
# Now we can lens our light
no_smooth_lensed_shape = no_smooth_lens_model_shape.image(no_smooth_lens_model_kwargs, shapelet_kwargs_list,
                                                          None, None)
no_smooth_lensed_shape_noise = no_smooth_lensed_shape+single_band.noise_for_model(no_smooth_lensed)

In [ ]:
# Plot the residual from the true model (i.e. without refitting)
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(no_smooth_lensed_shape_noise-source_lensed_shape_no_sub, origin='lower')
f.colorbar(im)
ax[0].set_title("No LS Residual (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(no_smooth_lensed_shape-source_lensed_shape_no_sub, origin='lower')
ax[1].set_title("No LS Residual (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

# Plot the residual with refitting
f, ax = plt.subplots(1, 2, figsize=(16, 5), sharex=False, sharey=False)
im = ax[0].matshow(source_lensed_shape_noise-source_lensed_shape_recon, origin='lower')
f.colorbar(im)
ax[0].set_title("Fit Residual (Noise)")
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].autoscale(False)

# Plot the residual no noise
im = ax[1].matshow(source_lensed_shape-source_lensed_shape_recon, origin='lower')
ax[1].set_title("Fit Residual (No Noise)")
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].autoscale(False)
plt.show()

## Using a COSMOS Source <a class="anchor" id="cosmos"></a>

As a quick proof of concept let's make some lenses using the cosmos galaxies. We want to integrate this functionality into baobab.

In [ ]:
# Load the cosmo galaxy examples
cosmos_dataset = np.load('../datasets/cosmos/cosmos_for_training.npz')

# Define the number of lenses of each type to output and the blurring to use
n_lenses = 3
image_dimension = len(cosmos_dataset['images'][0])
x, y = util.make_grid(numPix=image_dimension, deltapix=1)
photo_z = cosmos_dataset['catalog']['zphot']
noise_means = cosmos_dataset['catalog']['NOISE_MEAN']
sigma = 0
low_cut = 0.01
high_cut = 0.9

for i in range(2):
    if i == 0:
        print('Most light is not in the bulge')
    else:
        print('Most light is in the bulge')
    for nl in range(n_lenses):
        if i == 0:
            cosmo_gal = cosmo_dataset['images'][cosmos_dataset['catalog']['fit_dvc_btt']<low_cut][nl]
            print(photo_z[cosmos_dataset['catalog']['fit_dvc_btt']<low_cut][nl],
                  noise_means[cosmos_dataset['catalog']['fit_dvc_btt']<low_cut][nl])
        else:
            cosmo_gal = cosmo_dataset['images'][cosmos_dataset['catalog']['fit_dvc_btt']>high_cut][nl]
            print(photo_z[cosmos_dataset['catalog']['fit_dvc_btt']>high_cut][nl],
                  noise_means[cosmos_dataset['catalog']['fit_dvc_btt']>high_cut][nl])
        cosmo_gal = scipy.ndimage.filters.gaussian_filter(cosmo_gal, sigma, mode='nearest', truncate=6)

        # Turn the image in a single 1d array for shapelets
        image_1d = util.image2array(cosmo_gal) 

        # Specify our shapelet parameters
        n_max = 50 
        beta = 10

        # Decompose image and return the shapelet coefficients
        shapeletSet = ShapeletSet()
        coeff_gal = shapeletSet.decomposition(image_1d, x, y, n_max, beta, 1., center_x=0, center_y=0)
        image_reconstructed = shapeletSet.function(x, y, coeff_gal, n_max, beta, center_x=0, center_y=0)
        image_reconstructed_2d = util.array2image(image_reconstructed)

        # Now we can create our lensed image as before, but using a cosmos source
        cosmos_model_list = ['SHAPELETS']

        # We'll chose a different size for the source and set the amplitude to roughly match our desired magnitude
        source_beta = 0.2
        # For now I'm just chosing this by hand so the brightness is similar
        amp_coeff = coeff_gal*100
        cosmos_kwargs_list = [{'n_max': n_max, 'beta': source_beta, 'center_x': 0.0, 'center_y': 0.0, 'amp':amp_coeff}]
        cosmos_light_model = LightModel(cosmos_model_list)

        # Now we can make the image model
        cosmos_image_model = ImageModel(data_api.data_class, psf_model, complete_lens_model, cosmos_light_model, None, 
                                          None, kwargs_numerics=kwargs_numerics)

        # And finally we can generate the image
        cosmos_lensed_source = cosmos_image_model.image(complete_lens_model_kwargs, cosmos_kwargs_list, None, None)
        cosmos_lensed_source_noise = cosmos_lensed_source+single_band.noise_for_model(cosmos_lensed_source)

        # Plot the original source
        f, ax = plt.subplots(1, 3, figsize=(12, 5), sharex=False, sharey=False)
        im = ax[0].matshow(cosmo_gal, origin='lower')
        ax[0].set_title("Original Source")
        ax[0].get_xaxis().set_visible(False)
        ax[0].get_yaxis().set_visible(False)

        # Plot the reconstructed source
        im = ax[1].matshow(image_reconstructed_2d, origin='lower')
        ax[1].set_title("Reconstructed Source")
        ax[1].get_xaxis().set_visible(False)
        ax[1].get_yaxis().set_visible(False)

        # Plot the lensed reconstructed source
        im = ax[2].matshow(cosmos_lensed_source, origin='lower')
        ax[2].set_title("Lensed Reconstructed Source")
        ax[2].get_xaxis().set_visible(False)
        ax[2].get_yaxis().set_visible(False)
        plt.show()

In [ ]:
cosmos_dataset['catalog'].dtype

In [ ]:
cosmos_dataset['catalog']['NOISE_MEAN']

In [ ]:
plt.hist(cosmos_dataset['catalog']['fit_dvc_btt'])

In [ ]:
plt.imshow(cosmos_dataset['images'][cosmos_dataset['catalog']['fit_dvc_btt']<0.1][0])
plt.colorbar()

In [ ]:
# Start by fitting a realistic source using shapelets.
ngc_filename = os.path.join('ngc1300.jpg')
# Read data, this only works if you execute this notebook within the environment of the github repo!
ngc_data = imageio.imread(ngc_filename, as_gray=True, pilmode=None)

# Subtract the median of an edge of the image
median = np.median(ngc_data[:200, :200])
ngc_data -= median

# Resize the image to square size (add zeros at the edges of the non-square bits of the image)
nx, ny = np.shape(ngc_data)
n_min = min(nx, ny)
n_max = max(nx, ny)
ngc_square = np.zeros((n_max, n_max))
x_start = int((n_max - nx)/2.)
y_start = int((n_max - ny)/2.)
ngc_square[x_start:x_start+nx, y_start:y_start+ny] = ngc_data

# Convolve the image with a Gaussian convolution kernel of a few pixels
sigma = 5
ngc_conv = scipy.ndimage.filters.gaussian_filter(ngc_square, sigma, mode='nearest', truncate=6)

# Degrade the data
factor = 25 
numPix_large = int(len(ngc_conv)/factor)
n_new = int((numPix_large-1)*factor)
ngc_cut = ngc_conv[0:n_new,0:n_new]
x, y = util.make_grid(numPix=numPix_large-1, deltapix=1)  
ngc_data_resized = image_util.re_size(ngc_cut, factor)

# Turn the image in a single 1d array for shapelets
image_1d = util.image2array(ngc_data_resized) 

# Specify our shapelet parameters
n_max = 100 
beta = 10

# Decompose image and return the shapelet coefficients
shapeletSet = ShapeletSet()
coeff_ngc = shapeletSet.decomposition(image_1d, x, y, n_max, beta, 1., center_x=0, center_y=0)
image_reconstructed = shapeletSet.function(x, y, coeff_ngc, n_max, beta, center_x=0, center_y=0)

# View our 2D Image
image_reconstructed_2d = util.array2image(image_reconstructed)
plt.imshow(image_reconstructed_2d,origin='lower')
plt.title('Shapelet Source')
plt.colorbar()
plt.show()